# Instructions for Step 4

Jointly determine cause and effect segments using causal feature learning. While Step1 determined which variables are candidate causes, we here aim to determine the range of values of those.

In [3]:
from func import *

In [4]:
# Total cause list
# Hide internal variables name
cause_dict = {
    'ticket':{
        'pre1':[],
        'treat':[],
        'pre1treat':[]
    },
    'merch':{
        'pre1':[],
        'treat':[],
        'pre1treat':[],
    },
    'share':{
        'pre1':[],
        'treat':[],
        'pre1treat':[],
    },
    'stream':{
        'pre1':[],
        'treat':[],
        'pre1treat':[],
    },
}

In [5]:
raw_input = load_data('../Data/CausalFandom_main_data.pickle')


## 1. Prepare data

combine 8 potential causes data of 4 outcomes to put into XGBoost

## 1.1 Prepare raw data

In [ ]:
# combine 4 outcomes and 8 potential causes
stream_change = (raw_input[['streams_active_streams_following_four_weeks',
  'streams_programmed_streams_following_four_weeks']].sum(axis = 1) - raw_input[['streams_active_streams_pre_period_1',
  'streams_programmed_streams_pre_period_1']].sum(axis = 1)).rename('stream_change')

share_change = (raw_input['shares_following_four_weeks'] - raw_input['shares_pre_period_1']).rename('share_change')

ticket_change = (raw_input['tickets_following_four_weeks'] - raw_input['tickets_pre_period_1']).rename('ticket_change')

merch_change = (raw_input['merch_pre_period_1'] - raw_input['merch_following_four_weeks']).rename('merch_change')

outcome_colnames = ['share_change','ticket_change','stream_change','merch_change']

allposscaues = []

# Put 4 changes after all possible causes
combinexy_data = pd.concat([raw_input[allposscaues],stream_change, share_change, ticket_change, merch_change],axis=1)

## 1.2 Filtering data

1. Filter both zero ot outcome in pre-treatment period and post-treatment period
2. Filter extreme values
2. Apply log sampling to all outcomes' data

In [ ]:
# Remove data that are both zero in pre and post
idx_stream = (((raw_input['streams_active_streams_pre_period_1'] + raw_input['streams_programmed_streams_pre_period_1']) != 0 ) | ((raw_input['streams_active_streams_following_four_weeks'] + raw_input['streams_programmed_streams_following_four_weeks'] )!= 0 ))
idx_share = ((raw_input['shares_pre_period_1']!= 0 ) | (raw_input['shares_following_four_weeks'] != 0 ))
idx_merch = ((raw_input['merch_pre_period_1']!= 0 ) | (raw_input['merch_following_four_weeks'] != 0 ))
idx_ticket = ((raw_input['tickets_pre_period_1']!= 0 ) | (raw_input['tickets_following_four_weeks'] != 0 ))

streamdata = combinexy_data[idx_stream].reset_index(drop = True)
sharedata = combinexy_data[idx_share].reset_index(drop = True)
ticketdata = combinexy_data[idx_ticket].reset_index(drop = True)
merchdata = combinexy_data[idx_merch].reset_index(drop = True)


In [ ]:
# Apply llog sampling to all outcomes (further apply filtering on change of outcomes)
streamsample = sample_data(streamdata[abs(streamdata['stream_change']) < 50].reset_index(drop = True),'log',50,keyword='stream_change')
sharesample = sample_data(sharedata[abs(sharedata['share_change']) < 15].reset_index(drop = True),'log',200,keyword='share_change')
ticketsample = sampledata(ticketdata[abs(ticketdata['ticket_change']) < 15].reset_index(drop = True),'log',200,keyword='ticket_change')
merchsample = sampledata(merchdata[abs(merchdata['merch_change']) < 15].reset_index(drop = True),'log',200,keyword='merch_change')


In [ ]:
# Combine data of each outcome
subset_combineoutcome = pd.concat([streamsample, sharesample, ticketsample, merchsample], axis = 0).reset_index(drop = True)

# Filter data using bounds to exclude extreme values
idxoutlier = (abs(subset_combineoutcome['merch_change'])<5) & (abs(subset_combineoutcome['ticket_change'])<50) & (abs(subset_combineoutcome['share_change'])<15) & (subset_combineoutcome['stream_change']<50) & (abs(subset_combineoutcome['streams_active_streams_treatment_period'])<100) & (subset_combineoutcome['shares_treatment_period'] <30) & (subset_combineoutcome['tickets_treatment_period']<20) & (subset_combineoutcome['streams_programmed_streams_treatment_period']<100) & (subset_combineoutcome['streams_active_streams_pre_period_1']<100) & (subset_combineoutcome['streams_programmed_streams_pre_period_1']<100) & (subset_combineoutcome['shares_pre_period_1']<30)
subset1 = subset_combineoutcome[idxoutlier].reset_index(drop = True)

## 1.2 Perform scaling

In [ ]:
# standardized scale
scaled_data = subset1
for item in outcome_colnames:
    scaled_data[item] = (subset1[item]-subset1[item].mean(axis=0))/(subset1[item].std(axis = 0))

In [ ]:
# Visualise the distribution of stream change after scaling
plt.hist(scaled_data['stream_change'])

In [ ]:
# Whether to use a sampled data to reduce running time
# fracdata = scaled_data.sample(100000,random_state=727)
fracdata = scaled_data

## 1.3 Plot PCA on outcome and print linear combinations of PCs

In [ ]:
# Plot the all outcomes in PCA space (before clustering)
pca = PCA(n_components=4)
pca_result = pca.fit_transform(fracdata[outcome_colnames])
plt.scatter(pca_result[:,0],pca_result[:,1])
plt.show()

In [ ]:
# Print the linear combination of PC1 and PC2
pca_compos = pd.DataFrame(data = {
             'PCA1':pca.components_[0],
            'PCA2':pca.components_[1]
}, index= outcome_colnames)
print(pca_compos)
print(pca.explained_variance_ratio_)

## 2. Perform Causal Feature Learning (CFL)

1. Regression of x on y (Using XGBoost)
2. Cluster regression value f(x), to have x's causal class
3. Represent all y using 'knn format'
4. Cluster 'knn format' y to have y's causal class

## 2.1 Regression of x on y

In [ ]:
# Split train and test data
x_tr, x_te, y_tr, y_te = train_test_split(fracdata[allposscaues],fracdata[outcome_colnames], test_size=0.3, random_state=4242)

In [ ]:
# Train XGBoost model and test on the test dataset
model_lin = XGBRegressor()
model_lin.fit(x_tr,y_tr)
y_pred_lin = model_lin.predict(x_te)
print('MAE',mean_absolute_error(y_te,y_pred_lin))

In [ ]:
# Prepare data for regression
inputx = fracdata[allposscaues]
data_y = fracdata[outcome_colnames]
numofcluster1 = 4

# Regression
fx = model_lin.predict(inputx)
y_reg = fx

## 2.2 Cluster regression value f(x)

In [ ]:
#1. cluster on the f(x)
randomseed = int(time())%100*(int(time()*10)%10)
kmeans = KMeans(n_clusters = numofcluster1, random_state=randomseed+7, n_init = 100).fit(y_reg)
labelx = kmeans.labels_

## 2.3 Perform K nearest neighbors for each y

In [ ]:
# The kth neighbor to calculate the distance
kthneighbor = 4

neigh = NearestNeighbors(n_neighbors=kthneighbor)
n_dis_data = np.ones((len(data_y), 1))

for i in tqdm(range(numofcluster1)):
    data_y_clus = data_y[labelx == i]
    y_clus = data_y_clus
    neigh.fit(y_clus)
    distance, idx_neigh = neigh.kneighbors(data_y, return_distance=True)
    distance = distance[:, -1]
    distance = distance.reshape(-1, 1)
    n_dis_data = np.concatenate((n_dis_data, distance), axis = 1)

n_dis_data_clean = n_dis_data[:, 1:]


## 2.4 Perform clustering on new representation of y

In [ ]:
# Specify number of clusters of y
numofcluster2 = 4

kmeans = KMeans(n_clusters = numofcluster2, random_state= randomseed+2, n_init = 10).fit(n_dis_data_clean)

labely = kmeans.labels_.tolist()


## 3 Visualise clusters in PCA

1. Visualise clusters of y
2. Visualise clusters of x

## 3.1 Visualise cluster of y in PCA

In [ ]:
# Using the PCA result calculated before
Yplot = pca_result

colindex = 1
plt.figure(figsize = (8,6))
plt.title('Cluster Plot')
listran = list(set(labely))

for type in range(len(listran)):
    plt.scatter(Yplot[pd.Series(labely) == type][:,0], Yplot[pd.Series(labely) == type][:,1], label = f"""clus{type+1}""",marker = marker_list[type])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='upper right')
plt.show()

## 3.2 Perform PCA on x (before clustering)

In [ ]:
pca = PCA(n_components=4)
pca_result1 = pca.fit_transform(fracdata[allposscaues])
plt.scatter(pca_result1[:,0],pca_result1[:,1])
plt.show()

In [ ]:
# Print linear combinations of PC1 and PC2
pca_compos = pd.DataFrame(data = {
             'PCA1':pca.components_[0],
            'PCA2':pca.components_[1]
}, index= allposscaues)
print(pca_compos)
print('Explained variance ratio',pca.explained_variance_ratio_)


## 3.3 Visualise cluster of x in PCA

In [ ]:
# Using the PCA result calculated above
Yplot = pca_result1

colindex = 1
plt.figure(figsize = (8,6))
plt.title('Cluster Plot')
listran = list(set(labely))

for type in range(len(listran)):
    plt.scatter(Yplot[pd.Series(labelx) == type][:,0], Yplot[pd.Series(labelx) == type][:,1], label = f"""clus{type+1}""",marker = marker_list[type])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Plot the confusion matrix of each x towards each y
y_frac = confusion_matrix(labelx, labely)
pd.DataFrame(y_frac)


## 3 Within cluster analysis

Calculate the coefficients of each outcome's causes within each cluster (detailed calculation see report Sec. 6.4)

## 3.1 Calculate the coefficients of causes of each outcome within each cluster

In [ ]:
# Combine all the causes
causelistall = cause_dict['stream']['treat'] + cause_dict['stream']['pre1']

yname_list = ['stream','share','ticket','merch']

# Prepare dictionary to store result
resultdict = {}
for yname in yname_list:
    resultdict[yname] = {}
    for yla in range(numofcluster2):
        resultdict[yname][yla] = []

# Prepare dictionary to store size
sizedict = {}
for yname in yname_list:
    sizedict[yname] = {}
    for yla in range(numofcluster2):
        sizedict[yname][yla] = []

# Calculate coefficient of each cause and store it for further analysis
for yla in range(numofcluster2):
    # Get subset of the data
    xdatareg = inputx[pd.Series(labely) == yla][causelistall]
    ydatareg = data_y[pd.Series(labely) == yla]
    for yname in yname_list:
        youtcomename = yname+'_change'
        for xvar in cause_dict[yname]['treat']:
            # Require the xvar>0 and extract this coef only
            idx = xdatareg[xvar] > 0
            variable_dataset_x = xdatareg[idx]
            variable_dataset_y = ydatareg[idx]
            if len(variable_dataset_y) == 0:
                effect_var = 0
            else:
                # Using Linear regression to fit the data
                model_lin_part = LinearRegression().fit(variable_dataset_x,variable_dataset_y[youtcomename])
                # Extract the coefficient of corresponding causes from model
                effect_var = model_lin_part.coef_[causelistall.index(xvar)]

            resultdict[yname][yla].append(effect_var)
            sizedict[yname][yla].append(len(variable_dataset_y))


## 3.2 Visualise coefficients and sizes of data

In [ ]:
# Iterate outcomes to plot the heatmap of the coeffcients value
listoftable = []
for yname in yname_list:
    currentdict = resultdict[yname]
    datatable = pd.DataFrame(data = currentdict,index=cause_dict[yname]['treat'])
    listoftable.append(datatable)

    plt.figure(figsize=(datatable.shape[1] + 1,datatable.shape[0]))
    sns.heatmap(datatable, annot= True, linewidth=.3)
    plt.title(yname)
    plt.xlabel('cluster number')
    plt.show()

In [ ]:
# Iterate outcomes to plot the heatmap of the size of data when calculating each coeffcient
listoftablesize = []
for yname in yname_list:
    currentdict = sizedict[yname]
    datatable = pd.DataFrame(data = currentdict,index=cause_dict[yname]['treat'])
    listoftablesize.append(datatable)

    plt.figure(figsize=(datatable.shape[1] + 1,datatable.shape[0]))
    sns.heatmap(datatable, annot= True, linewidth=.3)
    plt.title(yname)
    plt.xlabel('cluster number')
    plt.show()

## 3.3 Scaling all coefficients in terms of certain cause (inner variable hidden) to find the relationship between coeffcients

In [ ]:
# Scaling all coefficients in terms of a cause [inner variable hidden]
for yname in yname_list:
    currentdict = resultdict[yname]
    datatable = pd.DataFrame(data = currentdict,index=cause_dict[yname]['treat'])
    multipletable = (datatable/datatable.loc['certain cause, inner variable hidden',:]).round(5)

    plt.figure(figsize=(datatable.shape[1] + 1,datatable.shape[0]))
    sns.heatmap(multipletable, annot= True, linewidth=.3)
    plt.title(yname)
    plt.xlabel('cluster number')
    plt.show()